In [ ]:
from pymongo import MongoClient
from pprint import pprint
import pandas as pd

In [ ]:
mongo = MongoClient(port=27017)

In [ ]:
db = mongo['uk_food']

In [ ]:
print(db.list_collection_names())

In [ ]:
establishments = db['establishments']

In [ ]:
query = {'scores.Hygiene': 20}

print(f" Number of documents in result: {establishments.count_documents(query)}")

results = establishments.find(query)

print('\n First Result:')
pprint(results[0])

In [ ]:
query = {'scores.Hygiene': 20}
results = establishments.find(query)

hygiene20DF = pd.DataFrame(results) 

print(f"Rows in DataFrame: {len(hygiene20DF)}")

hygiene20DF.head(10)

In [ ]:
query = {'LocalAuthorityName': {'$regex': 'London'}, 'RatingValue': {'$gte': '4' }}

print(f" Number of documents in result: {establishments.count_documents(query)}")

results = establishments.find(query)

print('\n First Result:')
pprint(results[0])

In [ ]:
query = {'LocalAuthorityName': {'$regex': 'London'}, 'RatingValue': {'$gte': '4' }}
results = establishments.find(query)
londonHighRatingsDF = pd.DataFrame(results)

print(f"Rows in DataFrame: {len(londonHighRatingsDF)}")

londonHighRatingsDF.head(10)

In [ ]:
degree_search = 0.01
latitude = 51.49014200
longitude = 0.08384000

query = {
    'RatingValue': '5',
    'geocode.latitude':{
        '$gte': latitude-degree_search,
        '$lte': latitude+degree_search
    },
    'geocode.longitude': {
        '$gte': longitude-degree_search,
        '$lte': longitude+degree_search
    }
}
sort = [('score.Hygiene',1)] 
limit = 5

pprint(list(establishments.find(query).sort(sort). limit(limit)))

In [ ]:
results = establishments.find(query).sort(sort). limit(limit)
restaurantNearPenangDF = pd.DataFrame(results)
restaurantNearPenangDF

In [ ]:
pipeline = [
    {'$match': {'scores.Hygiene':0}},
    {
        '$group':
        {
            '_id': '$LocalAuthorityName', 'count': {'$sum': 1}
        }
    },
    {'$sort': {'count': -1}}
] 

results = list(establishments.aggregate(pipeline))

print(f" Number of documents in result: {len(results)}")

pprint(results[0:10])

In [ ]:
resultsDF = pd.DataFrame(results)

print(f"Number of rows in DataFrame {len(resultsDF)}")

resultsDF = resultsDF.rename(columns= {'_id': "Local Authority"})

resultsDF.head(10)